In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv("data/train_data.csv", index_col="id")

In [3]:
print(train_data.shape)

(66125, 31)


In [4]:
train_data.head()

,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,...,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
id,,,,,,,,,,,,,,,,,,,,,
1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,soytuber_custom,...,hokusai_becchu,NaN,26.0,herocharger_replica,NaN,68.0,sharp_neo,NaN,31.0,1
2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,jetsweeper,...,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,quadhopper_black,...,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0
4,2019-11-11T14:00:00+00:00,5.0.1,regular,standard,nawabari,engawa,bamboo14mk1,NaN,336,splatroller_becchu,...,liter4k,NaN,189.0,promodeler_mg,NaN,194.0,hotblaster_custom,NaN,391.0,0
5,2019-12-14T06:00:00+00:00,5.0.1,gachi,standard,hoko,chozame,bold_7,x,299,hissen_hue,...,sputtery_hue,x,45.0,bucketslosher_soda,x,246.0,wakaba,x,160.0,1


In [5]:
np.sum(train_data['y'] == 1) / train_data['y'].count(), np.sum(train_data['y'] == 0) / train_data['y'].count()

(0.524703213610586, 0.47529678638941397)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66125 entries, 1 to 66125
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   period      66125 non-null  object 
 1   game-ver    66125 non-null  object 
 2   lobby-mode  66125 non-null  object 
 3   lobby       66125 non-null  object 
 4   mode        66125 non-null  object 
 5   stage       66125 non-null  object 
 6   A1-weapon   66125 non-null  object 
 7   A1-rank     51681 non-null  object 
 8   A1-level    66125 non-null  int64  
 9   A2-weapon   66125 non-null  object 
 10  A2-rank     51681 non-null  object 
 11  A2-level    66125 non-null  float64
 12  A3-weapon   66125 non-null  object 
 13  A3-rank     51681 non-null  object 
 14  A3-level    66125 non-null  float64
 15  A4-weapon   66074 non-null  object 
 16  A4-rank     51638 non-null  object 
 17  A4-level    66074 non-null  float64
 18  B1-weapon   66125 non-null  object 
 19  B1-rank     51681 non-nul

## 両チームの最大レベルは？

In [7]:
pd.concat([train_data['A1-level'], train_data['A2-level'], train_data['A3-level'], train_data['A4-level'], train_data['B1-level'], train_data['B2-level'], train_data['B3-level'], train_data['B4-level']]).max()

936.0

In [8]:
train_data['A1-level']

id
1        139
2        198
3        114
4        336
5        299
        ... 
66121     66
66122    204
66123    116
66124    210
66125    132
Name: A1-level, Length: 66125, dtype: int64

In [9]:
train_data.sort_values('A1-level', ascending=False)['A1-level'][:10]

id
39468    585
53809    585
19670    585
65461    585
61630    584
46759    584
48713    584
22386    584
41758    584
32279    584
Name: A1-level, dtype: int64

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
mlb = MultiLabelBinarizer()
mlb.fit([set(train_data['A1-weapon'].unique())])

MultiLabelBinarizer()

In [12]:
MultiLabelBinarizer(classes=None, sparse_output=False)

MultiLabelBinarizer()

In [13]:
def trans_weapon(df, columns=['A1-weapon', 'A2-weapon', 'A3-weapon', 'A4-weapon']):
    weapon = df.fillna('none') 
    weapon_binarized = mlb.transform(weapon[columns].values)
    return pd.DataFrame(weapon_binarized, columns=mlb.classes_)

def make_input_output(df, with_y=False):
    a_weapon = trans_weapon(df, ['A1-weapon', 'A2-weapon', 'A3-weapon', 'A4-weapon'])
    b_weapon = trans_weapon(df, ['B1-weapon', 'B2-weapon', 'B3-weapon', 'B4-weapon'])
    X = pd.concat([a_weapon, b_weapon], axis=1)
    if with_y:
        y = df['y']
        return X, y, a_weapon, b_weapon
    return X, a_weapon, b_weapon

In [14]:
X, y, a_weapon, b_weapon = make_input_output(train_data, with_y=True)

/Users/abekoh/.local/share/virtualenvs/splatoon_game_winner-RneJieyj/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:994: UserWarning: unknown class(es) ['none'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [15]:
X

,52gal,52gal_becchu,52gal_deco,96gal,96gal_deco,bamboo14mk1,bamboo14mk2,bamboo14mk3,barrelspinner,barrelspinner_deco,...,spygadget_sorella,squiclean_a,squiclean_b,squiclean_g,sshooter,sshooter_becchu,sshooter_collabo,variableroller,variableroller_foil,wakaba
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
66121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66125 entries, 0 to 66124
Columns: 278 entries, 52gal to wakaba
dtypes: int64(278)
memory usage: 140.2 MB


In [17]:
a_weapon

,52gal,52gal_becchu,52gal_deco,96gal,96gal_deco,bamboo14mk1,bamboo14mk2,bamboo14mk3,barrelspinner,barrelspinner_deco,...,spygadget_sorella,squiclean_a,squiclean_b,squiclean_g,sshooter,sshooter_becchu,sshooter_collabo,variableroller,variableroller_foil,wakaba
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
66121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
66123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
b_weapon

,52gal,52gal_becchu,52gal_deco,96gal,96gal_deco,bamboo14mk1,bamboo14mk2,bamboo14mk3,barrelspinner,barrelspinner_deco,...,spygadget_sorella,squiclean_a,squiclean_b,squiclean_g,sshooter,sshooter_becchu,sshooter_collabo,variableroller,variableroller_foil,wakaba
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
66121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X.shape

(66125, 278)

In [20]:
X['splatroller']

,splatroller,splatroller
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
66120,0,0
66121,0,0
66122,0,1
66123,0,0


In [21]:
X['splatroller']

,splatroller,splatroller
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
66120,0,0
66121,0,0
66122,0,1
66123,0,0


In [22]:
#scikit-learnライブラリをimport
import sklearn
from sklearn.ensemble import RandomForestClassifier

#線形回帰モデルのインスタンス化
model = RandomForestClassifier()

#予測モデルの作成
model.fit(X, y)

RandomForestClassifier()

In [23]:
list(reversed(sorted(list(zip(model.feature_importances_, X.columns)))))[:10]


[(0.01327033085371366, 'prime_becchu'),
 (0.01140002823371169, 'splatroller'),
 (0.011141020789345202, 'nzap85'),
 (0.011103798144396469, 'splatroller'),
 (0.010523531978804776, 'wakaba'),
 (0.010387055489998166, 'prime_becchu'),
 (0.01010844234934155, 'nzap89'),
 (0.009763712412097164, 'wakaba'),
 (0.009524947670835077, 'nzap89'),
 (0.008975088132846136, 'dualsweeper_custom')]

In [24]:
# accuracyの出力
from sklearn.metrics import accuracy_score
y_true = y
y_pred = model.predict(X)
print(accuracy_score(y_true, y_pred))

0.9999848771266541


In [25]:
path_test = "data/test_data.csv"
test_data = pd.read_csv(path_test, index_col='id')
test_data.head()

,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,A2-weapon,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
id,,,,,,,,,,,,,,,,,,,,,
1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,herospinner_replica,...,160,nautilus47,x,151.0,l3reelgun_d,x,213.0,nzap89,x,306.0
2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,nzap89,...,326,dualsweeper_custom,s+,289.0,prime_becchu,s+,147.0,splatroller,s+,156.0
3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,bold,...,76,momiji,s,232.0,nzap83,s,183.0,promodeler_pg,s,105.0
4,2019-12-30T04:00:00+00:00,5.0.1,gachi,standard,asari,devon,prime_becchu,s+,192,splatspinner_collabo,...,90,quadhopper_black,s+,119.0,soytuber,s+,172.0,prime,s+,70.0
5,2019-10-15T16:00:00+00:00,5.0.1,gachi,standard,yagura,anchovy,l3reelgun_d,x,267,rapid_becchu,...,259,sshooter_becchu,x,198.0,splatscope,x,50.0,screwslosher_becchu,x,287.0


In [26]:
test_X, _, _ = make_input_output(test_data)

/Users/abekoh/.local/share/virtualenvs/splatoon_game_winner-RneJieyj/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:994: UserWarning: unknown class(es) ['none'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [27]:
#テスト結果の出力
test_predicted = model.predict(test_X)
submit_df = pd.DataFrame({'y': test_predicted})
submit_df.index.name = 'id'
submit_df.to_csv('tuto_submission.csv')

In [28]:
train_data['game-ver'].unique()

array(['5.0.1'], dtype=object)

In [29]:
train_data['lobby-mode'].unique()

array(['regular', 'gachi'], dtype=object)

In [30]:
train_data['lobby'].unique()

array(['standard'], dtype=object)

In [31]:
train_data['mode'].unique()

array(['nawabari', 'hoko', 'yagura', 'area', 'asari'], dtype=object)

In [32]:
train_data['stage'].unique()

array(['sumeshi', 'arowana', 'ama', 'engawa', 'chozame', 'hokke',
       'gangaze', 'shottsuru', 'mutsugoro', 'mozuku', 'kombu', 'devon',
       'battera', 'tachiuo', 'zatou', 'mongara', 'anchovy', 'ajifry',
       'hakofugu', 'fujitsubo', 'manta', 'otoro', 'bbass'], dtype=object)

In [33]:
pd.concat([train_data['A1-level'], train_data['A2-level'], train_data['A3-level'], train_data['A4-level'], train_data['B1-level'], train_data['B2-level'], train_data['B3-level'], train_data['B4-level']]).max()

936.0